# Batch estimation of odometry parameters
## Experiments: 2018-07-03<br>
Erik Frisk (erik.frisk@liu.se)<br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

## Imports

In [ ]:
import sys
if not ('../python' in sys.path):
    sys.path.append('../python')

import matplotlib.pyplot as plt
import random
from calib_util import LoadLogDataPickle, BoxOff
from calib_util import QualisysSpeed, OdoSpeed, GetCalibrationSection
import numpy as np
import pandas as pd
from scipy.optimize import minimize

datadir = '../cascar_logs/20180703/'

In [ ]:
%matplotlib 

#from pylab import rcParams
#%matplotlib inline
#rcParams['figure.figsize']=np.array(rcParams['figure.figsize'])*1.8

# Define helper functions

In [ ]:
def SteerFunction(steer, c0, c1):
    return c0 + steer*c1

def EvalOdometry(segment, x0, c0, c1, r, L, W):
    
    odo_hat = np.zeros((len(segment), 3))
    odo_hat[0, :] = x0[0:3]

    for k in range(0, len(segment)-1):
        delta = SteerFunction(segment['steer'].values[k], c0, c1)
        ds = np.pi*2*r/10*1/(1-W/2/L*np.tan(delta))
        odo_hat[k+1, :] = odo_hat[k, :] + ds*np.array([np.cos(odo_hat[k, 2]),
                                                       np.sin(odo_hat[k, 2]),
                                                       1/L*np.tan(delta)])

    odo_hat = pd.DataFrame(
        np.hstack((segment['t'].values.reshape(-1, 1), odo_hat)),
        columns=['t', 'x', 'y', 'phi'])
    
    err = (segment[['x', 'y']].values-odo_hat[['x', 'y']].values)
    loss = 1/err.shape[0]*np.sqrt(np.sum(err[:, 0]**2 + err[:, 1]**2))    
    
    return odo_hat, loss

def EstimateOdometry_rLW(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, theta0[0], theta0[1], *theta)
        return l
    
    th0 = theta0[2:5]  # r, L, W
    res = minimize(loss_fcn, th0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':False, 'maxiter':2000})
    thopt = np.hstack(([c0, c1], res.x))
    return thopt, res

def EstimateOdometry_crW(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, theta[0], theta[1], theta[2],
                            theta0[3], theta[3])
        return l
    
    th0 = np.array(theta0)[np.array([0, 1, 2, 4])]  # c0, c1, r, W
    res = minimize(loss_fcn, th0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':False, 'maxiter':2000})
    thopt = np.hstack((res.x[[0, 1, 2]], [theta0[3], res.x[3]]))
    return thopt, res

def EstimateOdometry_all(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, *theta)
        return l
    res = minimize(loss_fcn, theta0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':False, 'maxiter':2000})
    return res.x, res

def DelaySteer(data, tau):
    data_dt = {}
    for ds in data.keys():
        data_dt[ds] = {'pos': data[ds]['pos'], 'odo': data[ds]['odo'].copy()}
        data_dt[ds]['odo']['steer_orig'] = data_dt[ds]['odo']['steer']
        data_dt[ds]['odo']['steer'] = np.interp(data_dt[ds]['odo']['t'], data_dt[ds]['odo']['t']+tau, data_dt[ds]['odo']['steer'])
    return data_dt

## Load data

In [ ]:
data = {}

pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_13_58_slow.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=True, synchronize=True)
odo = odo_l
v_qualisys = QualisysSpeed(pos)
v_odo = OdoSpeed(odo)
data['slow'] = {'pos': pos, 'odo': odo}

pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_14_1_fast.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=True, synchronize=False)
odo = odo_l
data['fast'] = {'pos': pos, 'odo': odo}
data['fast']['pos']['t'] = data['fast']['pos']['t'] - 1.9

tau = 275/1000
data = DelaySteer(data, tau)

Plot data and verify odometry/positioning synchronization

In [ ]:
plt.figure(10, clear=True)
plt.subplot(1, 2, 1)
plt.plot(data['slow']['pos']['x'], data['slow']['pos']['y'])
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Slow')
BoxOff()

plt.subplot(1, 2, 2)
plt.plot(data['slow']['pos']['t'], QualisysSpeed(data['slow']['pos']), 'b', label='v_pos')
plt.plot(data['slow']['odo']['t'], OdoSpeed(data['slow']['odo']), 'r', label='v_odo')
plt.legend()
plt.xlabel('t [s]')
plt.ylabel('speed [m/s]')
plt.title('Slow')
BoxOff()

plt.figure(11, clear=True)
plt.subplot(1, 2, 1)
plt.plot(data['fast']['pos']['x'], data['fast']['pos']['y'])
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Fast')
BoxOff()

plt.subplot(1, 2, 2)
plt.plot(data['fast']['pos']['t'], QualisysSpeed(data['fast']['pos']), 'b', label='v_pos')
plt.plot(data['fast']['odo']['t'], OdoSpeed(data['fast']['odo']), 'r', label='v_odo')
plt.legend()
plt.xlabel('t [s]')
plt.ylabel('speed [m/s]')
plt.title('Fast')
BoxOff()

# Prepare data

In [ ]:
T_data = {'fast': [5, 20], 'slow': [7, 35]}
dt = 2

d = {}
for ds in T_data.keys():
    t_int = np.arange(T_data[ds][0], T_data[ds][1], dt)
    d[ds] = [GetCalibrationSection(data[ds]['pos'], data[ds]['odo'], ti, ti+dt) for ti in t_int]

# Estimation

In [ ]:
c0 = 0.0202330878136394
c1 = 0.0016996712762432785
r = 78.0/1000/2
L = 28.0/100
W = 21.0/100 
theta0 = [c0, c1, r, L, W]

In [ ]:
th = {'slow': [], 'fast': []}
loss = {'slow': [], 'fast': []}
for ds in th.keys():
    print('Estimating in data set: ' + ds)
    print('-'*len(d[ds]))
    for odo, x0 in d[ds]:
        print('.', end='')
#        theta_opt, r = EstimateOdometry_all(odo, x0, theta0)
        theta_opt, r = EstimateOdometry_crW(odo, x0, theta0)
#        theta_opt, r = EstimateOdometry_rLW(odo, x0, theta0)

        th[ds].append(theta_opt)
        loss[ds].append(r.fun)
    print('')
    th[ds] = np.array(th[ds])
    loss[ds] = np.array(loss[ds])
print('Done!')

# Plot results

In [ ]:
plt.figure(10, clear=True)
plt.subplot(1, 2, 1)
plt.plot(loss['slow']/np.median(loss['slow']))
plt.xlabel('Estimation section')
plt.ylabel('Median normalized loss function')
plt.title('Dataset: slow')
BoxOff()

plt.subplot(1, 2, 2)
plt.plot(loss['fast']/np.median(loss['fast']))
plt.xlabel('Estimation section')
plt.ylabel('Median normalized loss function')
plt.title('Dataset: fast')
BoxOff()
plt.tight_layout()

In [ ]:
params = ['c0', 'c1', 'r', 'L', 'W']
scale = [1, 1, 100, 100, 100]
unit = ['', '', 'cm', 'cm', 'cm']
plt.figure(20, clear=True)
for k, (p, s, u) in enumerate(zip(params, scale, unit)):
    plt.subplot(3, 2, k+1)
    plt.plot(th['slow'][:, k]*s, 'b', label='slow')
    plt.plot(th['fast'][:, k]*s, 'r', label='fast')
    plt.xlabel('Estimation section')
    plt.ylabel(u)
    plt.title('Parameter: ' + p)
    BoxOff()
plt.legend(loc='upper right', bbox_to_anchor=(1.5, 1))
plt.tight_layout()

In [ ]:
plt.figure(30, clear=True)
plt.plot(th['slow'][0]/th['slow'][3], 'b', label='c0/L (slow)')
plt.plot(th['slow'][1]/th['slow'][3], 'b--', label='c1/L (slow)')
plt.plot(th['fast'][0]/th['fast'][3], 'r', label='c0/L (fast)')
plt.plot(th['fast'][1]/th['fast'][3], 'r--', label='c1/L (fast)')
plt.legend(loc='upper right')
BoxOff()

# Get median and evaluate

In [ ]:
th_result = np.median(np.vstack((th['slow'], th['fast'])), axis=0)

print('theta0   : %s' % theta0)
print('theta_opt: %s' % th_result)

In [ ]:
deviation = {'slow': [], 'fast': []}

plt.figure(60, clear=True)
for odo, x0 in d['slow']:
    odo_hat, _  = EvalOdometry(odo, x0, *th_result)
    plt.plot(odo['x'], odo['y'], 'b')
    plt.plot(odo_hat['x'], odo_hat['y'], 'r')
    delta = odo[['x', 'y']][-1:].values-odo_hat[['x', 'y']][-1:].values
    deviation['slow'].append(np.linalg.norm(delta))
deviation['slow'] = np.array(deviation['slow'])

plt.xlabel('x [m]')
plt.xlabel('y [m]')
plt.title('Dataset: slow, dt = %.1f s' % dt )

plt.figure(61, clear=True)
for odo, x0 in d['fast']:
    odo_hat, _  = EvalOdometry(odo, x0, *th_result)
    plt.plot(odo['x'], odo['y'], 'b')
    plt.plot(odo_hat['x'], odo_hat['y'], 'r')
    delta = odo[['x', 'y']][-1:].values-odo_hat[['x', 'y']][-1:].values
    deviation['fast'].append(np.linalg.norm(delta))
deviation['fast'] = np.array(deviation['fast'])

plt.xlabel('x [m]')
plt.xlabel('y [m]')
plt.title('Dataset: fast, dt = %.1f s' % dt );

plt.figure(62, clear=True)
plt.plot(deviation['slow']*100, 'b', label='slow')
plt.plot(deviation['fast']*100, 'r', label='fast')
plt.xlabel('Estimation section')
plt.ylabel('cm')
plt.title('End point deviation')
plt.legend()
BoxOff()

# Compare steering functions from circle experiments and driving experiments

In [ ]:
steer = np.array([-100, 100])
plt.figure(70, clear=True)
plt.plot(steer, 180/np.pi*SteerFunction(steer, theta0[0], theta0[1]), label='circle estimate')
plt.plot(steer, 180/np.pi*SteerFunction(steer, th_result[0], th_result[1]), label='driving estimate')
plt.xlabel('Steer [%]')
plt.ylabel('deg')
plt.title('Comparison, steering functions')
plt.legend()
plt.plot(plt.xlim(), [0, 0], 'k--', lw=0.75)
plt.plot([0, 0], plt.ylim(), 'k--', lw=0.75)
BoxOff()

The models differ, but the experiments from driving experiments is consistent with rough measurements on wheel angles on the vehicle.